In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun May 29 01:08:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive #Only if you are using Google Drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/My\ Drive/

Mounted at /content/gdrive
Mounted at /content/gdrive
/content/gdrive/My Drive


In [ ]:
cd /content/gdrive/MyDrive/content/img_dir

/content/gdrive/MyDrive/content/img_dir


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

from keras import applications
from keras.models import Model
import keras 
from keras.applications.vgg16 import VGG16 #Importing the VGG16 Model
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D,BatchNormalization
#from keras.applications import MobileNetV2
#from keras.optimizers import SGD
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input  

In [ ]:
training_dir = '/content/gdrive/MyDrive/content/img_dir/train'
validation_dir = '/content/gdrive/MyDrive/content/img_dir/test'
test_dir = '/content/gdrive/MyDrive/content/img_dir/test'

folders = glob(training_dir + '/*')
num_classes = len(folders)
print ('Total Classes = ' + str(num_classes))
print('Total training classes '+str(len(glob(training_dir + '/*'))))
print('Total test classes '+str(len(glob(test_dir + '/*'))))
print('Toral validation classes '+str(len(glob(validation_dir + '/*'))))

Total Classes = 10
Total training classes 10
Total test classes 10
Toral validation classes 10


Data option 1

In [ ]:
IMAGE_SIZE = [224,224]
BATCH_SIZE = 32

training_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,samplewise_center=True, samplewise_std_normalization=True)
training_generator = training_datagen.flow_from_directory(directory=training_dir,target_size = IMAGE_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, samplewise_center=True, samplewise_std_normalization=True)
validation_generator = validation_datagen.flow_from_directory(directory=test_dir, target_size = IMAGE_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')
training_generator.class_indices


Found 6969 images belonging to 10 classes.
Found 1747 images belonging to 10 classes.


{'air_conditioner': 0,
 'car_horn': 1,
 'children_playing': 2,
 'dog_bark': 3,
 'drilling': 4,
 'engine_idling': 5,
 'gun_shot': 6,
 'jackhammer': 7,
 'siren': 8,
 'street_music': 9}

Data Option 2

In [ ]:
# 255 batch size
IMAGE_SIZE = [224,224]
BATCH_SIZE = 32

training_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=40, 
                                      width_shift_range=0.3, height_shift_range=0.3, 
                                      shear_range=0.3, horizontal_flip=True, fill_mode="nearest",
                                      preprocessing_function=preprocess_input,
                                      samplewise_center=True, samplewise_std_normalization=True)
training_generator = training_datagen.flow_from_directory(directory=training_dir,target_size = IMAGE_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')
validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input, samplewise_center=True, samplewise_std_normalization=True)
validation_generator = validation_datagen.flow_from_directory(directory=test_dir, target_size = IMAGE_SIZE, batch_size = BATCH_SIZE, class_mode = 'categorical')
training_generator.class_indices

In [ ]:
vgg_model = VGG16(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False)  

for layer in vgg_model.layers:
      layer.trainable = False # Non trainable weights

# Create Dense Layers
# Add the last layers (Flatten and Dense layers) for our problem
x = Flatten()(vgg_model.output) 
x = Dense(num_classes, activation = 'softmax')(x)
x = Dropout(0.5)(x) # Dropout layer to reduce overfitting

transfer_model = Model(inputs = vgg_model.input, outputs = x)
# Compile model, for this we will be using ADAM optimiser to reach to the global minima while training our model
# learning_rate= 5e-5
#transfer_model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
transfer_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                       optimizer=tf.keras.optimizers.Adam(1e-3),
                       metrics=['accuracy'])

#transfer_model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#                       optimizer=tf.keras.optimizers.Adam(1e-3),
#                       metrics=['accuracy'])
#history = transfer_model.fit(X_train, y_train, batch_size = 1, epochs=50, validation_data=(X_test,y_test))

#transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
transfer_model.summary() #check summary of the model using this command


  #optimizer=tf.keras.optimizers.Adam(1e-3),
  #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [ ]:
# 8716 images, 32 batch size 
history = transfer_model.fit(training_generator,
                   steps_per_epoch =272, #Number of iterations = number of training images (49,225) / batch size (255)  
                   epochs = 10, 
                   validation_data = validation_generator,
                   validation_steps = 2, #same for validation data 300 validation images
                   shuffle = True) 
transfer_model.save('/content/gdrive/MyDrive/final/AEC_vgg16_us8k_272steps_10epochs_Param.h5')

Epoch 1/10
218/272 [=======================>......] - ETA: 8:22 - loss: nan - accuracy: 0.1725WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 2720 batches). You may need to use the repeat() function when building your dataset.


272/272 [==============================] - 2071s 8s/step - loss: nan - accuracy: 0.1725 - val_loss: nan - val_accuracy: 0.0938


In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

Tensorflow variant of VGG16

In [ ]:
batch_size = 32
img_height = 224
img_width = 224
#BATCH_SIZE = 255
IMG_SIZE = (224, 224)

#train_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/train'
#validation_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/test/'
data_dir = '/content/gdrive/MyDrive/content/img_dir/combined_data/all_labeled/'

vgg16 = tf.keras.applications.vgg16.VGG16(include_top=False, weights='imagenet')


feature_extractor_model = vgg16

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

class_names = np.array(train_ds.class_names)
print(class_names)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y)) # Where x—images, y—labels.

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
feature_extractor_layer = hub.KerasLayer(
    feature_extractor_model,
    input_shape=(224, 224, 3),
    trainable=False)

In [ ]:
feature_batch = feature_extractor_layer(image_batch)
print(feature_batch.shape)

In [ ]:
num_classes = len(class_names)

model = tf.keras.Sequential([
  feature_extractor_layer,
  tf.keras.layers.Dense(num_classes)
])

model.summary()

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(1e-3),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['acc'])

log = "/content/gdrive/MyDrive/content/img_dir/logs/vgg16/all_combined/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log, histogram_freq=1) # Enable histogram computation for every epoch.

In [ ]:
NUM_EPOCHS = 40

history = model.fit(train_ds,
                    validation_data=val_ds,
                    epochs=NUM_EPOCHS,
                    callbacks=tensorboard_callback)

In [ ]:
model.save('/content/gdrive/MyDrive/content/vgg16_all_combined_headless_final')
model.save('/content/gdrive/MyDrive/content/vgg16_all_combined_headless_final.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
 %reload_ext tensorboard

In [ ]:
%tensorboard --logdir="/content/gdrive/MyDrive/content/img_dir/logs/vgg16/all_combined/fit/"

In [ ]:
tf.keras.applications.vgg16.VGG16(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)